**ESCUELA DE INGENIERÍA MECATRÓNICA**

## `PROCESAMIENTO DIGITAL DE SEÑALES E IMÁGENES`

### `Integrantes:`

* Baltazar Julca Cristhian
* Garay Alcantara Alex
* Gonzales Bañes Ricardo Danfer
```
Práctica 7B: Filtrado en la frecuencia
```

###  1. Revise e implemente la interfaz basandose en el snippet de la pagina siguiente.

In [38]:
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np
import cv2

def escalar_imagen(image, max_width):
    original_height, original_width = image.shape
    ratio = max_width / original_width
    height = int(original_height * ratio)
    return cv2.resize(image, (max_width, height))

def mostrar_imagen(image, label, max_width):
    imagen_resized = escalar_imagen(image, max_width)
    imagen_tk = ImageTk.PhotoImage(image=Image.fromarray(imagen_resized))
    label.config(image=imagen_tk)
    label.image = imagen_tk

def actualizar_filtro(valor):
    mostrar_imagen(img, label_matriz, 300)
    mostrar_imagen(img, label_g_real, 300)

image_path = "MachuPicchu.jpg"  # Ruta de la imagen a mostrar
img = cv2.imread(image_path, 0)

ventana = tk.Toplevel()
ventana.title("Mostrar Imagen y Matriz con Tkinter")

label_imagen = tk.Label(ventana)
mostrar_imagen(img, label_imagen, 300)
label_imagen.grid(row=0, column=0, padx=10, pady=10)

label_matriz = tk.Label(ventana)
label_matriz.grid(row=0, column=1, padx=10, pady=10)

label_g_real = tk.Label(ventana)
label_g_real.grid(row=0, column=2, padx=10, pady=10)

slider_n = tk.Scale(ventana, from_=0, to=1, resolution=0.1, orient=tk.HORIZONTAL,
                    label="Valor de n", command=actualizar_filtro)
slider_n.set(2)
slider_n.grid(row=1, column=0, columnspan=3, pady=10)

ventana.mainloop()


###  2. Implemente un algoritmo 
Que le permita tener una interfaz con un slider para controlar el nivel de suavizado de una imagen. El slider controlara el tamaño
del filtro frecuencia que se aplicará. Puede aplicar cualquiera de los filtros
pasabajos mostrados. Se debe poder ver la imagen original, el filtro frecuencial
aplicado y el resultado.

In [6]:
# Filtro pasa bajo gaussiano con slider

import tkinter as tk
from PIL import Image, ImageTk
import numpy as np
import cv2

def filter_pab(img , tam):
    fil, col = img.shape
    img_padded = np.pad(img,((0,fil),(0,col)),'constant',constant_values=((0, 0),(0,0)))

    f, c = np.ogrid[0:2*fil,0:2*col]

    D =  np.sqrt( (f-fil)**2 + (c-col)**2 )
    D0 = (tam*fil) #Varia
    H_pa_gauss = np.exp(-(D**2)/(2*(D0**2)))

    img_fft = np.fft.fft2(img_padded)
    img_fft_shift = np.fft.fftshift(img_fft)

    G_fft = img_fft_shift * H_pa_gauss

    G_fft_ishift = np.fft.ifftshift(G_fft)
    g = np.fft.ifft2(G_fft_ishift)
    g_real = np.real(g)[0:fil, 0:col]

    return H_pa_gauss*255, g_real

def escalar_imagen(image, max_width):
    original_height, original_width = image.shape
    ratio = max_width / original_width
    height = int(original_height * ratio)
    return cv2.resize(image, (max_width, height))

def mostrar_imagen(image, label, max_width):
    imagen_resized = escalar_imagen(image, max_width)
    imagen_tk = ImageTk.PhotoImage(image=Image.fromarray(imagen_resized))
    label.config(image=imagen_tk)
    label.image = imagen_tk

def actualizar_filtro(valor):
    img_H, img_g = filter_pab(img,float(valor))
    mostrar_imagen(img_H, label_matriz, 300)
    mostrar_imagen(img_g, label_g_real, 300)

image_path = "emma.jpg"  # Ruta de la imagen a mostrar
img = cv2.imread(image_path, 0)

ventana = tk.Toplevel()
ventana.title("Mostrar Imagen y Matriz con Tkinter")

label_imagen = tk.Label(ventana)
mostrar_imagen(img, label_imagen, 300)
label_imagen.grid(row=0, column=0, padx=10, pady=10)

label_matriz = tk.Label(ventana)
label_matriz.grid(row=0, column=1, padx=10, pady=10)

label_g_real = tk.Label(ventana)
label_g_real.grid(row=0, column=2, padx=10, pady=10)

slider_n = tk.Scale(ventana, from_=0.1, to=1, resolution=0.1, orient=tk.HORIZONTAL,
                    label="Valor de n", command=actualizar_filtro)
slider_n.set(1)
slider_n.grid(row=1, column=0, columnspan=3, pady=10)

ventana.mainloop()


### 3. Implemente un algoritmo 
Que le permita tener una interfaz para controlar la
ganancia y tamaño de un filtro de alto aumento. Se debe poder ver la imagen
original, el filtro frecuencial aplicado y el resultado.

In [37]:
# HIGH BOOST FILTERING WHITH SLIDER
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np
import cv2

def filter_pab(img , tam, A):
    fil, col = img.shape
    img_padded = np.pad(img,((0,fil),(0,col)),'constant',constant_values=((0, 0),(0,0)))

    f, c = np.ogrid[0:2*fil,0:2*col]

    D =  np.sqrt( (f-fil)**2 + (c-col)**2 )
    D0 = (tam*fil) #Varia
    H_pa_gauss = 1 - np.exp(-(D**2)/(2*(D0**2)))

    H_hb_gauss = (A-1)+H_pa_gauss 
    
    img_fft = np.fft.fft2(img_padded)
    img_fft_shift = np.fft.fftshift(img_fft)

    G_fft = img_fft_shift * H_hb_gauss 

    G_fft_ishift = np.fft.ifftshift(G_fft)
    g = np.fft.ifft2(G_fft_ishift)
    g_real = np.real(g)[0:fil, 0:col]
     
    return H_hb_gauss*255, g_real

def escalar_imagen(image, max_width):
    original_height, original_width = image.shape
    ratio = max_width / original_width
    height = int(original_height * ratio)
    return cv2.resize(image, (max_width, height))

def mostrar_imagen(image, label, max_width):
    imagen_resized = escalar_imagen(image, max_width)
    imagen_tk = ImageTk.PhotoImage(image=Image.fromarray(imagen_resized))
    label.config(image=imagen_tk)
    label.image = imagen_tk

def actualizar_filtro(valor):
    img_H, img_g = filter_pab(img,float(slider_n.get()),float(slider_n1.get()))
    mostrar_imagen(img_H, label_matriz, 300)
    mostrar_imagen(img_g, label_g_real, 300)

image_path = "emma.jpg"  # Ruta de la imagen a mostrar
img = cv2.imread(image_path, 0)

ventana = tk.Toplevel()
ventana.title("Mostrar Imagen y Matriz con Tkinter")

label_imagen = tk.Label(ventana)
mostrar_imagen(img, label_imagen, 300)
label_imagen.grid(row=0, column=0, padx=10, pady=10)

label_matriz = tk.Label(ventana)
label_matriz.grid(row=0, column=1, padx=10, pady=10)

label_g_real = tk.Label(ventana)
label_g_real.grid(row=0, column=2, padx=10, pady=10)

slider_n = tk.Scale(ventana, from_=0.1, to=1, resolution=0.1, orient=tk.HORIZONTAL,
                    label="Valor de n", command=actualizar_filtro)
slider_n.set(0.1)
slider_n.grid(row=1, column=0,columnspan=2, pady=10)

slider_n1 = tk.Scale(ventana, from_=0.1, to=3, resolution=0.1, orient=tk.HORIZONTAL,
                    label="Valor de A", command=actualizar_filtro)
slider_n1.set(1.5)
slider_n1.grid(row=1, column=1,columnspan=2, pady=10)

ventana.mainloop()